In [1]:
#Importing the lib!raries
import pandas as pd
from sklearn.model_selection import train_test_split

In [2]:
#mount the file through goole drive and read it and store it in the pandas data frame
from google.colab import drive
drive.mount('/content/gdrive/')

Mounted at /content/gdrive/


In [3]:
#Changing to the current directory pointing to the folder
%cd '/content/gdrive/MyDrive/Email_Text_Summarization'

/content/gdrive/MyDrive/Email_Text_Summarization


In [4]:
#Reading the file and displaying the first few rows
dialog_df=pd.read_json('dialogsum.train.jsonl',lines=True)
dialog_df.head()

,fname,dialogue,summary,topic
0,train_0,"#Person1#: Hi, Mr. Smith. I'm Doctor Hawkins. ...","Mr. Smith's getting a check-up, and Doctor Haw...",get a check-up
1,train_1,"#Person1#: Hello Mrs. Parker, how have you bee...",Mrs Parker takes Ricky for his vaccines. Dr. P...,vaccines
2,train_2,"#Person1#: Excuse me, did you see a set of key...",#Person1#'s looking for a set of keys and asks...,find keys
3,train_3,#Person1#: Why didn't you tell me you had a gi...,#Person1#'s angry because #Person2# didn't tel...,have a girlfriend
4,train_4,"#Person1#: Watsup, ladies! Y'll looking'fine t...",Malik invites Nikki to dance. Nikki agrees if ...,dance


In [5]:
!pip install --upgrade pip

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 24.7 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 23.1.2
    Uninstalling pip-23.1.2:
      Successfully uninstalled pip-23.1.2


In [6]:
!pip install pytorch-lightning==1.5.10

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 527.7/527.7 kB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 952.4/952.4 kB 44.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 805.2/805.2 kB 43.6 MB/s eta 0:00:00
DEPRECATION: pytorch-lightning 1.5.10 has a non-standard dependency specifier torch>=1.7.*. pip 24.0 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of pytorch-lightning or contact the author to suggest that they release a version with a conforming dependency specifiers. Discussion can be found at https://github.com/pypa/pip/issues/12063
  Attempting uninstall: setuptools
    Found existing installation: setuptools 67.7.2
    Uninstalling setuptools-67.7.2:
      Successfully uninstalled setuptools-67.7.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ipython 7.34.0 requires 

In [7]:
!pip install --upgrade simplet5

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 22.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.5/3.5 MB 89.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 897.5/897.5 kB 52.5 MB/s eta 0:00:00
  Created wheel for simplet5: filename=simplet5-0.1.4-py3-none-any.whl size=6868 sha256=c4fa5f8da8d5470fb6216c1a1f8fbc5e3b408842b74e8cee539a0172453b0666
  Stored in directory: /root/.cache/pip/wheels/b4/7d/af/743765400878438a7593f13f89fdf4004dcde0f2a8e6cb6684
Successfully built simplet5
DEPRECATION: pytorch-lightning 1.5.10 has a non-standard dependency specifier torch>=1.7.*. pip 24.0 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of pytorch-lightning or contact the author to suggest that they release a version with a conforming dependency specifiers. Discussion can be found at https://github.com/pypa/pip/issues/12063
  Attempting uninstall: transformers
    Found e

In [8]:
#Reading only the dialog and summary column from one dataframe to another dataframe to train my model.
train_df = dialog_df[['dialogue','summary']]
#Iam sampling only the first 5000 rows into the data set for training purpose
sample_train_df=train_df.head(5000)

In [9]:
# simpleT5 model expects dataframe to have 2 columns: "source_text" and "target_text".
#I have named the summary column as the tearget column which would be the end result.
#I have named the dialogue column as the source text for the model to predict the necessary summary.
sample_train_df = sample_train_df.rename(columns={"summary":"target_text", "dialogue":"source_text"})
sample_train_df = sample_train_df[['source_text', 'target_text']]


In [10]:
# T5 model expects a task related prefix: since it is a summarization task, we will add a prefix "summarize: "
#As mentioned above this code mainly takes the task name summarize as prefix and is added before all the sample rows
sample_train_df['source_text'] = "summarize: " + sample_train_df['source_text']
sample_train_df.head(3)

,source_text,target_text
0,"summarize: #Person1#: Hi, Mr. Smith. I'm Docto...","Mr. Smith's getting a check-up, and Doctor Haw..."
1,"summarize: #Person1#: Hello Mrs. Parker, how h...",Mrs Parker takes Ricky for his vaccines. Dr. P...
2,"summarize: #Person1#: Excuse me, did you see a...",#Person1#'s looking for a set of keys and asks...


In [11]:
#The base model remains T5 and I will be downloading the entire model simpleT5 in this code.
from simplet5 import SimpleT5

model = SimpleT5()
model.from_pretrained(model_type="t5", model_name="t5-base")

INFO:pytorch_lightning.utilities.seed:Global seed set to 42


Downloading:   0%|          | 0.00/773k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.32M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.18k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/850M [00:00<?, ?B/s]

In [13]:
#I will be training the model with the train data set after making the necessary steps previously.
model.train(train_df=sample_train_df,
           eval_df=sample_train_df[:100],
           source_max_token_len=128,
           target_max_token_len=50,
           batch_size=8, max_epochs=5, use_gpu=True)

INFO:pytorch_lightning.utilities.distributed:GPU available: True, used: True
INFO:pytorch_lightning.utilities.distributed:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.distributed:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.accelerators.gpu:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.callbacks.model_summary:
  | Name  | Type                       | Params
-----------------------------------------------------
0 | model | T5ForConditionalGeneration | 222 M 
-----------------------------------------------------
222 M     Trainable params
0         Non-trainable params
222 M     Total params
891.614   Total estimated model params size (MB)


Validation sanity check: 0it [00:00, ?it/s]

/usr/local/lib/python3.10/dist-packages/pytorch_lightning/trainer/data_loading.py:132: UserWarning: The dataloader, val_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 8 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(
INFO:pytorch_lightning.utilities.seed:Global seed set to 42
/usr/local/lib/python3.10/dist-packages/pytorch_lightning/trainer/data_loading.py:132: UserWarning: The dataloader, train_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 8 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


Training: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

In [14]:
#I am checking the loss value and I am looking at the least loss value.
! (cd outputs; ls)

simplet5-epoch-0-train-loss-1.5761-val-loss-1.2998
simplet5-epoch-0-train-loss-1.5778-val-loss-1.3074
simplet5-epoch-1-train-loss-1.3516-val-loss-1.1165
simplet5-epoch-1-train-loss-1.355-val-loss-1.1292
simplet5-epoch-2-train-loss-1.2147-val-loss-0.9845
simplet5-epoch-2-train-loss-1.2151-val-loss-0.9906
simplet5-epoch-3-train-loss-1.094-val-loss-0.8505
simplet5-epoch-3-train-loss-1.0962-val-loss-0.8581
simplet5-epoch-4-train-loss-0.9907-val-loss-0.7411
simplet5-epoch-4-train-loss-0.9975-val-loss-0.742


In [15]:
#load the trained model from the local output folder for inferencing. Here Iam getting the least loss value from the
#outputs folder and passing it as the path for the model inorder to predict the summary which is more precise.
model.load_model("t5","outputs/simplet5-epoch-4-train-loss-0.9907-val-loss-0.7411", use_gpu=True)

In [16]:
text_to_summarize="""summarize: Twitter’s interim resident grievance officer for India has stepped down, leaving the micro-blogging site without a grievance official as mandated by the new IT rules to address complaints from Indian subscribers, according to a source.

The source said that Dharmendra Chatur, who was recently appointed as interim resident grievance officer for India by Twitter, has quit from the post.

The social media company’s website no longer displays his name, as required under Information Technology (Intermediary Guidelines and Digital Media Ethics Code) Rules 2021.

Twitter declined to comment on the development.

The development comes at a time when the micro-blogging platform has been engaged in a tussle with the Indian government over the new social media rules. The government has slammed Twitter for deliberate defiance and failure to comply with the country’s new IT rules.
"""
model.predict(text_to_summarize)

['Dharmendra Chatur, who was recently appointed as interim resident grievance officer for India by Twitter, has quit from the post. The micro-blogging site has been engaged in a tussle with the Indian government over the new social media rules.']

In [17]:
text_to_summarize="""summarize: Vaccination and safety measures such as wearing face masks are essential when it comes to fighting the Delta Plus coronavirus variant, World Health Organization (WHO) representative to Russia Melita Vujnovic said.

"Vaccination plus masks, because just a vaccine is not enough with 'Delta Plus'. We need to make an effort over a short period of time, otherwise there would be a lockdown," Vujnovic said on the Soloviev Live YouTube show.

She explained that vaccination is essential because it lowers the probability of spreading the virus and lowers the risks of severe disease. However, "additional measures" will probably be required as well, Vujnovic warned.

Earlier in June, the WHO included the Delta variant in its list of coronavirus variants of concern as the strain had become prevalent and has caused a resurgence of COVID-19 cases in some countries, including Russia. India has also reported multiple cases of the Delta Plus strain, which was first discovered in March.
"""
model.predict(text_to_summarize)

['Melita Vujnovic, a WHO representative to Russia, says vaccination and safety measures are essential for fighting the Delta Plus coronavirus variant. Vujnovic also suggests wearing face masks.']

In [18]:


text_to_summarize="""summarize: Travellers vaccinated with Covishield may not be eligible for the European Union’s ‘Green Pass’ that will be available for use from July 1. Many EU member states have started issuing the digital “vaccine passport” that will enable Europeans to move freely for work or tourism. The immunity passport will serve as proof that a person has been vaccinated against the coronavirus disease (Covid-19), or recently tested negative for the virus, or has the natural immunity built up from earlier infection.Covishield, a version of AstraZeneca Covid vaccine manufactured by Pune-based Serum Institute of India (SII), has not been approved by the EMA for the European market. The EU green pass will only recognise the Vaxzervria version of the AstraZeneca vaccine that is manufactured in the UK or other sites around Europe.
"""
model.predict(text_to_summarize)



['Travellers vaccinated with Covishield may not be eligible for the European Union’s ‘Green Pass’ that will be available for use from July 1.']

In [19]:
test=train_df[6000:6800]
test.shape

(800, 2)

In [21]:
pip install rouge

DEPRECATION: pytorch-lightning 1.5.10 has a non-standard dependency specifier torch>=1.7.*. pip 24.0 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of pytorch-lightning or contact the author to suggest that they release a version with a conforming dependency specifiers. Discussion can be found at https://github.com/pypa/pip/issues/12063


In [24]:
!pip install transformers[torch]

DEPRECATION: pytorch-lightning 1.5.10 has a non-standard dependency specifier torch>=1.7.*. pip 24.0 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of pytorch-lightning or contact the author to suggest that they release a version with a conforming dependency specifiers. Discussion can be found at https://github.com/pypa/pip/issues/12063


##Evaluation Metrics using Rouge Score

In [43]:
generated_summaries[:5]
flattened_list = [item for sublist in generated_summaries for item in sublist]
flattened_list[:5]

["Steven Smith opens an account with #Person1#'s assistance.",
 'Alice wants to hire an interior decorator to look over her house. Jack suggests buying potted plants and a chandelier in the living room.',
 'Tom likes jazz while #Person1# likes rock music.',
 '#Person1# and #Person2# think serious pollution in modern cities goes from bad to worse.',
 "#Person1# and #Person2# cannot ignore a woman's suggestion."]

In [45]:
target_summaries[:5]

['Steven wants to open an account. #Person1# guides him to fill a form and to queue up.',
 'Jack has a nice house but many things have been going wrong lately. Then Alice offers some suggestions about his home decoration.',
 '#Person1# likes Jazz while Tom likes rock. They talk about Wanton Marsalis and the Cranberries.',
 '#Person1# and #Person2# think people should take actions to protect the environment.',
 '#Person2# thinks #Person1# has no guts to go against his wife.']

In [44]:

# Prepare lists to store scores
rouge_1_scores = []
rouge_2_scores = []
rouge_l_scores = []


# Calculate ROUGE scores for each pair of generated summary and target summary
for generated_summary, target_summary in zip(flattened_list, target_summaries):
    scores = rouge.get_scores(generated_summary, target_summary)

    # Extract ROUGE-1, ROUGE-2, and ROUGE-L scores
    rouge_1_scores.append(scores[0]['rouge-1']['f'])
    rouge_2_scores.append(scores[0]['rouge-2']['f'])
    rouge_l_scores.append(scores[0]['rouge-l']['f'])

# Calculate average scores
avg_rouge_1 = sum(rouge_1_scores) / len(rouge_1_scores)
avg_rouge_2 = sum(rouge_2_scores) / len(rouge_2_scores)
avg_rouge_l = sum(rouge_l_scores) / len(rouge_l_scores)

# Print or use the scores as needed
print(f'Average ROUGE-1 Score: {avg_rouge_1}')
print(f'Average ROUGE-2 Score: {avg_rouge_2}')
print(f'Average ROUGE-L Score: {avg_rouge_l}')


Average ROUGE-1 Score: 0.4590525178252598
Average ROUGE-2 Score: 0.2061086606842975
Average ROUGE-L Score: 0.422953347404966
